In [1]:
import json
import networkx as nx
import pandas as pd
import numpy as np

### final summarization algorithm

In [2]:
# load clusters
with open("../data/astro0/astro0_decomposed_roberta_clusters.json") as fp:
    clusters = json.load(fp)
clusters

{'0': [32, 81],
 '1': [39, 85],
 '2': [17, 18, 20, 21, 38, 67],
 '3': [1, 2, 3, 48],
 '4': [82],
 '5': [33, 34],
 '6': [86, 87],
 '7': [54, 68],
 '8': [79, 80],
 '9': [78],
 '10': [64],
 '11': [77],
 '12': [74, 75],
 '13': [83],
 '14': [84],
 '15': [69],
 '16': [70, 71],
 '17': [66],
 '18': [56],
 '19': [51],
 '20': [61],
 '21': [36],
 '22': [60],
 '23': [27],
 '24': [49],
 '25': [42],
 '26': [31],
 '27': [76],
 '28': [73],
 '29': [57],
 '30': [63],
 '31': [65],
 '32': [62],
 '33': [35],
 '34': [72],
 '35': [58],
 '36': [55],
 '37': [40],
 '38': [16],
 '39': [24],
 '40': [41],
 '41': [50],
 '42': [26],
 '43': [7],
 '44': [30],
 '45': [46],
 '46': [53],
 '47': [22],
 '48': [25],
 '49': [8],
 '50': [15],
 '51': [6],
 '52': [4],
 '53': [10],
 '54': [12],
 '55': [29],
 '56': [11],
 '57': [23],
 '58': [43],
 '59': [14],
 '60': [45],
 '61': [52],
 '62': [59],
 '63': [9],
 '64': [47],
 '65': [44],
 '66': [0],
 '67': [5],
 '68': [13],
 '69': [19],
 '70': [37],
 '71': [28]}

In [3]:
# load inter-cluster distances
intercluster_dist = pd.read_csv("../data/astro0/astro0_decomposed_roberta_clusterMatrix.csv", header=None)
intercluster_dist.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.000000,0.460077,0.643337,0.587756,0.496971,0.492482,0.414728,0.492909,0.488133,0.485273,...,0.482410,0.499528,0.462201,0.499491,0.407001,0.666479,0.478474,0.494741,0.489900,0.488647
1,0.460077,0.000000,0.520432,0.451625,0.474576,0.675942,0.455312,0.629915,0.698304,0.722739,...,0.481723,0.499407,0.479619,0.645681,0.421498,0.442398,0.479124,0.447941,0.494378,0.493064
2,0.643337,0.520432,0.000000,0.500876,0.552524,0.566930,0.503410,0.730273,0.528661,0.639558,...,0.485424,0.499452,0.476062,0.711230,0.554892,0.469265,0.495814,0.492871,0.498495,0.498309
3,0.587756,0.451625,0.500876,0.000000,0.681033,0.492950,0.494025,0.343024,0.497913,0.494197,...,0.490091,0.499479,0.438295,0.498716,0.611256,0.492028,0.562030,0.492076,0.499047,0.498358
4,0.496971,0.474576,0.552524,0.681033,0.000000,0.495674,0.683693,0.860534,0.617868,0.891487,...,0.442565,0.499455,0.497795,0.820072,0.489331,0.491849,0.597681,0.496999,0.498410,0.498095


In [13]:
# load indexed sentences
sent_df = pd.read_csv("../data/astro0/astro0_decomposed_indexedSentences.csv")
sent_df.head()

,Unnamed: 0,0
0,0,"People with Virgo rising tend to be practical,..."
1,1,You have an ingenious mind.
2,2,You have an active mind.
3,3,You have an alert mind.
4,4,Gaining knowledge and putting it to good use a...


In [14]:
sent_df = sent_df.rename(columns={'0': 'sent'})
sent_df = sent_df.drop(['Unnamed: 0'], axis='columns')
sent_df.head()

,sent
0,"People with Virgo rising tend to be practical,..."
1,You have an ingenious mind.
2,You have an active mind.
3,You have an alert mind.
4,Gaining knowledge and putting it to good use a...


In [15]:
len(sent_df)

88

In [16]:
display(clusters['0'], clusters['3'])

[32, 81]

[1, 2, 3, 48]

In [18]:
# load pairwise distance matrix for sentence pairs.
sentence_dist = pd.read_csv("../data/astro0/astro0_decomposed_roberta_pairwiseMatrix.csv", index_col="first_index")
sentence_dist.head()

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
first_index,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.457454,0.890627,0.122736,0.343647,0.303749,0.494094,0.498978,0.428174,0.499487,...,0.490302,0.497844,0.497375,0.401196,0.489331,0.993485,0.491715,0.494522,0.378094,0.391084
1,0.457454,0.000000,0.032994,0.130562,0.498068,0.495861,0.498606,0.486919,0.480689,0.499531,...,0.485482,0.497580,0.498825,0.486510,0.874905,0.987602,0.485783,0.417588,0.497986,0.493121
2,0.890627,0.032994,0.000000,0.002692,0.456897,0.492376,0.498840,0.481217,0.465977,0.499415,...,0.496183,0.495881,0.499339,0.612499,0.667300,0.994634,0.498959,0.493914,0.491291,0.494619
3,0.122736,0.130562,0.002692,0.000000,0.458032,0.484349,0.498572,0.487478,0.456454,0.499436,...,0.496204,0.497620,0.499115,0.769684,0.424280,0.998456,0.497800,0.486022,0.490840,0.493866
4,0.343647,0.498068,0.456897,0.458032,0.000000,0.440961,0.499034,0.499095,0.831255,0.499499,...,0.452073,0.498734,0.499442,0.989376,0.861346,0.499028,0.490917,0.481579,0.367555,0.460451


In [23]:
A = sent_df[sent_df.sent == 'You have an ingenious mind.'].index[0]
B = sent_df[sent_df.sent == 'You have an active mind.'].index[0]
sentence_dist.iloc[A][B]

0.03299373033807409

In [33]:
notable_pairs = list()
for clust1 in intercluster_dist.index:
    for clust2 in intercluster_dist.columns:
        if intercluster_dist.loc[clust1][clust2] > 0.95:
            notable_pairs.append((clust1, clust2))

In [34]:
print(len(notable_pairs))
notable_pairs

228


[(0, 22),
 (1, 60),
 (3, 13),
 (3, 40),
 (4, 11),
 (4, 18),
 (4, 22),
 (4, 35),
 (4, 46),
 (4, 57),
 (6, 11),
 (6, 34),
 (7, 13),
 (7, 23),
 (7, 44),
 (7, 68),
 (9, 11),
 (9, 60),
 (9, 63),
 (10, 34),
 (10, 57),
 (10, 60),
 (11, 4),
 (11, 6),
 (11, 9),
 (11, 24),
 (11, 25),
 (11, 31),
 (11, 41),
 (11, 42),
 (11, 44),
 (11, 50),
 (11, 52),
 (11, 53),
 (11, 56),
 (11, 58),
 (11, 59),
 (11, 66),
 (11, 67),
 (11, 68),
 (13, 3),
 (13, 7),
 (13, 24),
 (13, 25),
 (13, 32),
 (13, 35),
 (13, 46),
 (13, 49),
 (13, 51),
 (13, 56),
 (13, 57),
 (13, 60),
 (13, 61),
 (13, 66),
 (13, 67),
 (14, 60),
 (16, 60),
 (18, 4),
 (18, 60),
 (19, 22),
 (19, 57),
 (19, 65),
 (20, 57),
 (20, 60),
 (22, 0),
 (22, 4),
 (22, 19),
 (22, 42),
 (22, 67),
 (23, 7),
 (23, 49),
 (23, 51),
 (23, 61),
 (24, 11),
 (24, 13),
 (24, 34),
 (24, 57),
 (24, 60),
 (25, 11),
 (25, 13),
 (25, 60),
 (26, 60),
 (29, 31),
 (29, 51),
 (29, 52),
 (29, 66),
 (29, 67),
 (30, 60),
 (31, 11),
 (31, 29),
 (31, 57),
 (31, 60),
 (32, 13),
 (32,

In [36]:
for pair in notable_pairs:
    print(f'clusters: {pair}')
    for index in clusters[str(pair[0])]:
        print(sent_df.iloc[index]['sent'])
    print()
    for index in clusters[str(pair[1])]:
        print(sent_df.iloc[index]['sent'])
    print('======\n')

clusters: (0, 22)
Security is very important to you.
Comfort is important to you.

Debate and argument appeal to you.

clusters: (1, 60)
you do not give up or let go of things or people easily.
Guard against possessiveness, jealousy, and taking the easy way out in your relationships.

Nothing or no one is ever gone if they leave their memory behind.

clusters: (3, 13)
You have an ingenious mind.
You have an active mind.
You have an alert mind.
If you have Mercury in Gemini, you have an extremely adaptable, energetic, active, alert, curious, and versatile mind.

At times you can be lazy, hoping or expecting someone else to do the dirty work.

clusters: (3, 40)
You have an ingenious mind.
You have an active mind.
You have an alert mind.
If you have Mercury in Gemini, you have an extremely adaptable, energetic, active, alert, curious, and versatile mind.

You are basically an introvert.

clusters: (4, 11)
you hate getting your hands dirty.

You enjoy indulging yourself.

clusters: (4, 18)

In [37]:
for num, cluster in clusters.items():
    print(len(cluster))

2
2
6
4
1
2
2
2
2
1
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [48]:
from nltk.tokenize import RegexpTokenizer

def choose_longest(sentences):
    tokenizer = RegexpTokenizer(r'\w+')
    max_len = 0
    final_sent = None
    for sent in sentences:
        tokens = tokenizer.tokenize(sent)
        if len(tokens) > max_len:
            max_len = len(sent)
            final_sent = sent
    return final_sent

from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS
def choose_salient(sentences):
    """
    
    """
    lxr = LexRank(sentences, stopwords=STOPWORDS['en'])
    top_sentence = lxr.get_summary(sentences[0], summary_size=1, threshold=.1)
    return top_sentence[0]
    

def summarize(clusters, intersentence, sentences, word_limit=90, coherence_threshold=0.6, redundancy_threshold=0.3):
    """
    :param clusters: sorted (descending in num of sentences) dict of clusters {cluster_num: list_of_sentence_indices}
    :param intersentence: Pandas DataFrame of precomputed inter-sentence distance in pairwise matrix form.
    :param sent_df: Pandas DataFrame of indexed sentences.
    :param word_limit: maximum num of words for summary.
    :param coherence_threshold: maximum distance allowed for a sentence pair within the summary. valid values between 0-1.
    :param redundancy_threshold: minimum distance allowed for a sentence pair within the summary. valid values between 0-1.
    
    We need a redundancy_threshold because the clustering algorithm does not always manage to group 
    redundant sentences together?
    """
    if coherence_threshold <= redundancy_threshold:
        raise Exception
    
    tokenizer = RegexpTokenizer(r'\w+')
    summary = list()
    word_count = 0
    for key, clust in clusters.items():
        sentences = [sent_df.iloc[idx]['sent'] for idx in clust]
        
        #selected = choose_longest(clust)
        selected = choose_salient([sentences])
        selected_idx = sent_df[sent_df.sent == selected].index[0]
        tokens = tokenizer.tokenize(selected)
        if word_count + len(tokens) < word_limit:
            max_dist = 0
            min_dist = 1
            for sent_idx in summary:
                dist = intersentence.iloc[sent_idx][selected_idx]
                if dist > max_dist:
                    max_dist = dist
                if dist < min_dist:
                    min_dist = dist
            if max_dist < coherence_threshold and min_dist > redundancy_threshold:
                summary.append(selected_idx)
                word_count = word_count + len(tokens)
        else:
            break
    return summary

In [49]:
# test lexrank algorithm.
sentences = [sent_df.iloc[idx]['sent'] for idx in clusters['2']]
choose_salient([sentences])

'Mentally restless, you may change jobs or locations frequently, or do work that involves movement, travel, and variety.'

In [57]:
summary = summarize(clusters, sentence_dist, sent_df, word_limit=90, coherence_threshold=0.55, redundancy_threshold=0.3)
print("no. of sentences in summary: ", len(summary))
print("no. of sentences in source: ", len(sentence_dist.index.values))
print()
for sent_idx in summary:
    print(f"{sent_idx}: {sent_df.iloc[sent_idx]['sent']}")
print()
for sent_idx in sorted(summary):
    print(f"{sent_idx}: {sent_df.iloc[sent_idx]['sent']}")

no. of sentences in summary:  7
no. of sentences in source:  88

81: Comfort is important to you.
85: Guard against possessiveness, jealousy, and taking the easy way out in your relationships.
82: you hate getting your hands dirty.
80: it is too easy for you to put too much value in material things.
75: You are tremendously responsive to physical appearances.
83: At times you can be lazy, hoping or expecting someone else to do the dirty work.
84: Self-control needs to be developed and added to the qualifications you already possess -warmth, charm, sympathy, artistic.

75: You are tremendously responsive to physical appearances.
80: it is too easy for you to put too much value in material things.
81: Comfort is important to you.
82: you hate getting your hands dirty.
83: At times you can be lazy, hoping or expecting someone else to do the dirty work.
84: Self-control needs to be developed and added to the qualifications you already possess -warmth, charm, sympathy, artistic.
85: Guard a

In [61]:
sentence_dist.iloc[81][82]

0.4978859601053069

In [62]:
# find sentences that contradict with the summary
coherence_threshold = 0.55
contradictory_set = set()
for sent in summary:
    for other in sentence_dist.loc[sent].index:
        if sentence_dist.loc[sent][other] >= coherence_threshold:
            if other not in contradictory_set:
                contradictory_set.add(other)
                print(other)

2
3
4
5
8
17
24
38
42
45
48
49
52
53
60
64
65
67
74
76
6
23
26
33
34
44
54
56
62
72
77
78
1
13
35
51
58
66
68
79
87
39
22
0
11


In [63]:
# find sentences that are redundant with the summary
redundancy_threshold = 0.3
redundant_set = set()
for sent in summary:
    for other in sentence_dist.loc[sent].index:
        if sentence_dist.loc[sent][other] <= redundancy_threshold:
            if other not in redundant_set:
                redundant_set.add(other)
                print(other)

32
81
39
85
82
79
80
74
75
83
84


In [64]:
contradictory_set & redundant_set

{'39', '74', '79'}